# Join 2023 stop and search data to LSOA boundaries
- Follow same process as 03_LSOA for historic data 
- load 2025_LSOA counts and merge

In [11]:
from pathlib import Path
import geopandas as gpd


zip_path = Path("../data/raw/statistical-gis-boundaries-london.zip")

shp_inside_zip = "statistical-gis-boundaries-london/ESRI/LSOA_2011_London_gen_MHW.shp"



lsoa = gpd.read_file(f"zip://{zip_path}!{shp_inside_zip}")

# Check
print(lsoa.head())
print(lsoa.crs)


    LSOA11CD                   LSOA11NM   MSOA11CD                  MSOA11NM  \
0  E01000001        City of London 001A  E02000001        City of London 001   
1  E01000002        City of London 001B  E02000001        City of London 001   
2  E01000003        City of London 001C  E02000001        City of London 001   
3  E01000005        City of London 001E  E02000001        City of London 001   
4  E01000006  Barking and Dagenham 016A  E02000017  Barking and Dagenham 016   

     LAD11CD               LAD11NM    RGN11CD RGN11NM  USUALRES  HHOLDRES  \
0  E09000001        City of London  E12000007  London      1465      1465   
1  E09000001        City of London  E12000007  London      1436      1436   
2  E09000001        City of London  E12000007  London      1346      1250   
3  E09000001        City of London  E12000007  London       985       985   
4  E09000002  Barking and Dagenham  E12000007  London      1703      1699   

   COMESTRES  POPDEN  HHOLDS  AVHHOLDSZ  \
0          0 

In [12]:
#OK need to convert the CRS to work with lat long


lsoa = lsoa.to_crs("EPSG:4326")

In [13]:
#import stop and search data 

import pandas as pd

processed_dir = Path("../data/processed")
stop_search_file = processed_dir / "2023_stop_search_aggregated.csv"

stop_and_search_2023_data = pd.read_csv(stop_search_file)

print(stop_and_search_2023_data.head())


                        Type                       Date  \
0              Person search  2023-01-01T00:00:00+00:00   
1              Person search  2023-01-01T00:01:00+00:00   
2              Person search  2023-01-01T00:01:00+00:00   
3  Person and Vehicle search  2023-01-01T00:02:00+00:00   
4  Person and Vehicle search  2023-01-01T00:02:00+00:00   

   Part of a policing operation  Policing operation   Latitude  Longitude  \
0                         False                 NaN  51.459946  -0.093261   
1                         False                 NaN  51.528765  -0.216088   
2                         False                 NaN  51.510468  -0.130769   
3                         False                 NaN  51.502641  -0.000790   
4                         False                 NaN  51.460105   0.113566   

  Gender Age range                             Self-defined ethnicity  \
0   Male     18-24                    Other ethnic group - Not stated   
1   Male     18-24  Black/African/Ca

In [14]:
#Convert dataframe to geodatafram

import geopandas as gpd
from shapely.geometry import Point

gdf = gpd.GeoDataFrame(
    stop_and_search_2023_data,
    geometry=[Point(xy) for xy in zip(stop_and_search_2023_data.Longitude, stop_and_search_2023_data.Latitude)],
    crs="EPSG:4326"
)


In [15]:
# Add LSOA data to geodataframe

joined = gpd.sjoin(
    gdf,
    lsoa,
    how="left",
    predicate="within"
)

In [16]:
#Check

print(joined[["LSOA11CD"]].head())

    LSOA11CD
0  E01004070
1  E01000586
2  E01004734
3  E01001667
4  E01000388


In [17]:
#Create counts for each area 

lsoa_counts = (
    joined
    .groupby("LSOA11CD")
    .size()
    .reset_index(name="stop_search_count")
)


In [18]:
# Add the counts to the polygons 

lsoa_map_2023 = lsoa.merge(
    lsoa_counts,
    on="LSOA11CD",
    how="left"
).fillna(0)


In [19]:
print(lsoa_map_2023.head())

    LSOA11CD                   LSOA11NM   MSOA11CD                  MSOA11NM  \
0  E01000001        City of London 001A  E02000001        City of London 001   
1  E01000002        City of London 001B  E02000001        City of London 001   
2  E01000003        City of London 001C  E02000001        City of London 001   
3  E01000005        City of London 001E  E02000001        City of London 001   
4  E01000006  Barking and Dagenham 016A  E02000017  Barking and Dagenham 016   

     LAD11CD               LAD11NM    RGN11CD RGN11NM  USUALRES  HHOLDRES  \
0  E09000001        City of London  E12000007  London      1465      1465   
1  E09000001        City of London  E12000007  London      1436      1436   
2  E09000001        City of London  E12000007  London      1346      1250   
3  E09000001        City of London  E12000007  London       985       985   
4  E09000002  Barking and Dagenham  E12000007  London      1703      1699   

   COMESTRES  POPDEN  HHOLDS  AVHHOLDSZ  \
0          0 

In [21]:
lsoa_map_2023 = lsoa_map_2023.rename(columns={"stop_search_count" : "stop_search_c_2023"})

In [22]:
print(lsoa_map_2023.head())

    LSOA11CD                   LSOA11NM   MSOA11CD                  MSOA11NM  \
0  E01000001        City of London 001A  E02000001        City of London 001   
1  E01000002        City of London 001B  E02000001        City of London 001   
2  E01000003        City of London 001C  E02000001        City of London 001   
3  E01000005        City of London 001E  E02000001        City of London 001   
4  E01000006  Barking and Dagenham 016A  E02000017  Barking and Dagenham 016   

     LAD11CD               LAD11NM    RGN11CD RGN11NM  USUALRES  HHOLDRES  \
0  E09000001        City of London  E12000007  London      1465      1465   
1  E09000001        City of London  E12000007  London      1436      1436   
2  E09000001        City of London  E12000007  London      1346      1250   
3  E09000001        City of London  E12000007  London       985       985   
4  E09000002  Barking and Dagenham  E12000007  London      1703      1699   

   COMESTRES  POPDEN  HHOLDS  AVHHOLDSZ  \
0          0 

In [23]:
# load 2025 data 

geo_stop_search_2025 = gpd.read_file("../data/processed/lsoa_stop_search.gpkg")

print(geo_stop_search_2025.head())

    LSOA11CD                   LSOA11NM   MSOA11CD                  MSOA11NM  \
0  E01000001        City of London 001A  E02000001        City of London 001   
1  E01000002        City of London 001B  E02000001        City of London 001   
2  E01000003        City of London 001C  E02000001        City of London 001   
3  E01000005        City of London 001E  E02000001        City of London 001   
4  E01000006  Barking and Dagenham 016A  E02000017  Barking and Dagenham 016   

     LAD11CD               LAD11NM    RGN11CD RGN11NM  USUALRES  HHOLDRES  \
0  E09000001        City of London  E12000007  London      1465      1465   
1  E09000001        City of London  E12000007  London      1436      1436   
2  E09000001        City of London  E12000007  London      1346      1250   
3  E09000001        City of London  E12000007  London       985       985   
4  E09000002  Barking and Dagenham  E12000007  London      1703      1699   

   COMESTRES  POPDEN  HHOLDS  AVHHOLDSZ  stop_search_cou

In [24]:
#check crs

print(geo_stop_search_2025.crs)
print(lsoa_map_2023.crs)

EPSG:4326
EPSG:4326


In [25]:
geo_stop_search_2025 = geo_stop_search_2025.rename(columns={'stop_search_count': 'stop_search_count_2025'}
)

In [29]:
merged_stop_search_2023_2025 = geo_stop_search_2025.merge(lsoa_map_2023[["LSOA11CD", "stop_search_c_2023"]], how="left", on=["LSOA11CD"])

print(merged_stop_search_2023_2025.head())

    LSOA11CD                   LSOA11NM   MSOA11CD                  MSOA11NM  \
0  E01000001        City of London 001A  E02000001        City of London 001   
1  E01000002        City of London 001B  E02000001        City of London 001   
2  E01000003        City of London 001C  E02000001        City of London 001   
3  E01000005        City of London 001E  E02000001        City of London 001   
4  E01000006  Barking and Dagenham 016A  E02000017  Barking and Dagenham 016   

     LAD11CD               LAD11NM    RGN11CD RGN11NM  USUALRES  HHOLDRES  \
0  E09000001        City of London  E12000007  London      1465      1465   
1  E09000001        City of London  E12000007  London      1436      1436   
2  E09000001        City of London  E12000007  London      1346      1250   
3  E09000001        City of London  E12000007  London       985       985   
4  E09000002  Barking and Dagenham  E12000007  London      1703      1699   

   COMESTRES  POPDEN  HHOLDS  AVHHOLDSZ  stop_search_cou

In [34]:
#calcualte absolute chnage 

merged_stop_search_2023_2025['abs_change'] = (merged_stop_search_2023_2025['stop_search_count_2025'] - merged_stop_search_2023_2025['stop_search_c_2023'])

In [35]:
merged_stop_search_2023_2025.to_file('../data/processed/lsoa_change_2023_2025.gpkg')

In [36]:
merged_stop_search_2023_2025.describe()

,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,stop_search_count_2025,stop_search_c_2023,abs_change
count,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000,4835.000000
mean,1690.577249,1669.844881,20.732368,95.937497,675.526991,2.519090,25.956153,26.063702,-0.107549
std,263.566401,257.715521,80.966217,61.223649,133.601312,0.390042,56.583086,66.938171,41.887738
min,985.000000,878.000000,0.000000,1.200000,404.000000,1.400000,0.000000,0.000000,-1372.000000
25%,1530.000000,1512.000000,0.000000,51.950000,590.500000,2.300000,5.000000,4.000000,-5.000000
50%,1654.000000,1639.000000,0.000000,83.300000,653.000000,2.500000,12.000000,11.000000,1.000000
75%,1817.000000,1798.000000,11.000000,128.400000,750.000000,2.700000,26.000000,25.000000,8.000000
max,4933.000000,3063.000000,2172.000000,684.700000,1236.000000,4.100000,1383.000000,1966.000000,464.000000
